In [ ]:
import tensorflow as tf
import pandas as pd
import json
import configparser
from pathlib import Path
import os

from sklearn.metrics import mean_absolute_percentage_error as MAPE
    
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling1D

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')

data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]
look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses previstos

In [ ]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano', 'salario', 'razao'], axis=1, inplace=True)
cesta = cesta / 1000

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[:-1:, :look_back:].values
y_train = cesta.iloc[:-1:, look_back::].values
X_val = cesta.iloc[-1::, :look_back:].values
y_val = cesta.iloc[-1::, look_back::].values
y_val2 = cesta.iloc[-1::, look_back::].values.tolist()

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(Conv1D(filters=24, kernel_size=3, input_shape=(look_back,1), activation='tanh'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(15, activation='tanh'))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_CNN_{objeto.lower()}_{regiao.lower()}_{forecast_horizon}_meses.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

# Prever 3 meses

In [ ]:
"""number_iter = 5

cross_validate_mapes = [[] for i in range(number_iter)]

for j in range(number_iter):
    i = 0
    #definir i < 30 para validação e i < 36 para teste
    while i < 36:

        #Carrega os dados
        cesta = pd.read_excel(path)
        cesta.drop(['ano', 'salario', 'razao'], axis=1, inplace=True)
        cesta = cesta / 1000

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        #Validação
        #X_train = cesta.iloc[:124 + i:, :look_back:].values
        #y_train = cesta.iloc[:124 + i:, look_back::].values
        #X_val = cesta.iloc[124 + i: 124 + i + 6:, :look_back:].values
        #y_val = cesta.iloc[124 + i: 124 + i + 6, look_back::].values
        #y_val2 = cesta.iloc[124 + i: 124 + i + 6:, look_back::].values.tolist()

        #Teste
        X_train = cesta.iloc[:154 + i:, :look_back:].values
        y_train = cesta.iloc[:154 + i:, look_back::].values
        X_val = cesta.iloc[154 + i: 154 + i + 6:, :look_back:].values
        y_val = cesta.iloc[154 + i: 154 + i + 6:, look_back::].values
        y_val2 = cesta.iloc[154 + i: 154 + i + 6:, look_back::].values.tolist()

        i += 6
        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(Conv1D(filters=24, kernel_size=3, input_shape=(look_back,1), activation='tanh'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(15, activation='tanh'))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False, validation_data=(X_val, y_val))
        forecast = model.predict(X_val, batch_size=1)

        cross_validate_mapes[j].append(MAPE(y_val2, forecast) * 100)"""

In [ ]:
"""print("cross_validate_mapes: ", cross_validate_mapes)
cross_validate_means = []

for current_mapes in cross_validate_mapes.copy():
    mean = 0
    for mape in current_mapes:
        mean += mape

    mean /= len(current_mapes)
    cross_validate_means.append(mean)

if len(cross_validate_means) >= 3:
    cross_validate_means.sort()
    cross_validate_means.pop()
    del cross_validate_means[0]

print("cross_validate_means: ", cross_validate_means)

general_mean = 0
for mean in cross_validate_means:
    general_mean += mean
general_mean /= len(cross_validate_means)

print("MAPE = ", round(general_mean, 3))"""